In [1]:
import cirq

In [2]:
# define the length and width of the grid.
length = 3

# define qubits on the grid.
qubits = cirq.GridQubit.square(length)

print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


In [3]:
circuit = cirq.Circuit()
circuit.append(cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0)
circuit.append(cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1)

print(circuit)

(0, 0): ───H───

(0, 1): ───X───

(0, 2): ───H───

(1, 0): ───X───

(1, 1): ───H───

(1, 2): ───X───

(2, 0): ───H───

(2, 1): ───X───

(2, 2): ───H───


In [4]:
def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""

    # Define the gate once and then re-use it for each Operation.
    rot = cirq.XPowGate(exponent=half_turns)

    # Create an X rotation Operation for each qubit in the grid.
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

# Create the circuit using the rot_x_layer generator
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


In [5]:
import random
def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length - 1, length)
    # links within a column
    jc = rand2d(length, length - 1)
    return (h, jr, jc)

h, jr, jc = random_instance(3)
print(f'transverse fields: {h}')
print(f'row j fields: {jr}')
print(f'column j fields: {jc}')

transverse fields: [[-1, -1, -1], [1, 1, -1], [-1, -1, -1]]
row j fields: [[-1, -1, 1], [1, 1, 1]]
column j fields: [[1, -1], [1, -1], [1, -1]]


In [6]:
def prepare_plus_layer(length):
    for i in range(length):
        for j in range(length):
            yield cirq.H(cirq.GridQubit(i, j))

In [7]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

In [8]:
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    cz_gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i + 1, j)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i, j + 1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

In [9]:
def initial_step(length):
    yield prepare_plus_layer(length)

def one_step(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
    length = len(h)
    yield rot_z_layer(h, h_half_turns)
    yield rot_11_layer(jr, jc, j_half_turns)
    yield rot_x_layer(length, x_half_turns)

h, jr, jc = random_instance(3)

circuit = cirq.Circuit()  
circuit.append(initial_step(len(h)))
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
print(circuit)

                       ┌──────┐   ┌──────────┐   ┌──────┐   ┌──────┐   ┌──────┐
(0, 0): ───H───X───────────────────@──────────────X──────────@──────────X^0.1─────────────────────────────────────────────────
                                   │                         │
(0, 1): ───H───Z^0.2─────@─────────┼─────────────────────────@^0.3──────@─────────X^0.1───────────────────────────────────────
                         │         │                                    │
(0, 2): ───H───Z^0.2────X┼─────────┼────@─────────X─────────────────────@^0.3─────X^0.1───────────────────────────────────────
                         │         │    │
(1, 0): ───H───Z^0.2────X┼─────────@^0.3┼─────────X──────────@──────────@─────────X^0.1───────────────────────────────────────
                         │              │                    │          │
(1, 1): ───H─────────────@^0.3─────X────┼──────────@─────────┼────X─────@^0.3─────X───────────@───────X───────X^0.1───────────
                                

In [10]:
simulator = cirq.Simulator()
circuit = cirq.Circuit()
circuit.append(initial_step(len(h)))
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))
results = simulator.run(circuit, repetitions=100)
print(results.histogram(key='x'))

Counter({175: 3, 399: 2, 301: 2, 30: 2, 415: 2, 59: 2, 269: 2, 139: 2, 371: 2, 21: 2, 31: 2, 140: 1, 283: 1, 380: 1, 45: 1, 58: 1, 17: 1, 271: 1, 130: 1, 256: 1, 333: 1, 280: 1, 329: 1, 138: 1, 158: 1, 270: 1, 84: 1, 508: 1, 477: 1, 381: 1, 145: 1, 456: 1, 187: 1, 13: 1, 402: 1, 111: 1, 179: 1, 189: 1, 506: 1, 291: 1, 411: 1, 216: 1, 427: 1, 397: 1, 434: 1, 331: 1, 345: 1, 501: 1, 107: 1, 467: 1, 34: 1, 99: 1, 286: 1, 323: 1, 341: 1, 285: 1, 146: 1, 79: 1, 53: 1, 303: 1, 222: 1, 279: 1, 332: 1, 221: 1, 462: 1, 343: 1, 281: 1, 376: 1, 94: 1, 465: 1, 478: 1, 313: 1, 155: 1, 81: 1, 347: 1, 298: 1, 211: 1, 160: 1, 115: 1, 439: 1, 282: 1, 479: 1, 68: 1, 67: 1, 297: 1, 203: 1, 277: 1, 312: 1})


In [11]:
import numpy as np

def energy_func(length, h, jr, jc):
    def energy(measurements):
        # Reshape measurement into array that matches grid shape.
        meas_list_of_lists = [measurements[i * length:(i + 1) * length]
                              for i in range(length)]
        # Convert true/false to +1/-1.
        pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

        tot_energy = np.sum(pm_meas * h)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
        return tot_energy
    return energy
print(results.histogram(key='x', fold_func=energy_func(3, h, jr, jc)))

Counter({1: 19, -3: 19, 3: 16, -1: 14, 7: 10, 9: 6, 5: 6, 11: 4, -5: 4, -7: 1, -15: 1})


In [12]:
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(3, h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions
print(f'Value of the objective function {obj_func(results)}')

Value of the objective function 1.52


In [13]:
import sympy
circuit = cirq.Circuit()
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
circuit.append(initial_step(len(h)))
circuit.append(one_step(h, jr, jc, alpha, beta, gamma))
circuit.append(cirq.measure(*qubits, key='x'))
print(circuit)

                        ┌────────┐   ┌──────────────┐   ┌────────┐   ┌────────┐   ┌────────┐
(0, 0): ───H───X──────────────────────@──────────────────X────────────@────────────X^alpha───────────────────────────────────────────────────────────M('x')───
                                      │                               │                                                                              │
(0, 1): ───H───Z^beta─────@───────────┼───────────────────────────────@^gamma──────@───────────X^alpha───────────────────────────────────────────────M────────
                          │           │                                            │                                                                 │
(0, 2): ───H───Z^beta────X┼───────────┼──────@───────────X─────────────────────────@^gamma─────X^alpha───────────────────────────────────────────────M────────
                          │           │      │                                                                                  

In [14]:
resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.3, 'gamma': 0.7})
resolved_circuit = cirq.resolve_parameters(circuit, resolver)

In [15]:
sweep = (cirq.Linspace(key='alpha', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='beta', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='gamma', start=0.1, stop=0.9, length=5))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
for result in results:
    print(result.params.param_dict, obj_func(result))

OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.1)]) -0.48
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.30000000000000004)]) 0.96
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.5)]) 0.24
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.7000000000000001)]) -1.22
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.9)]) -0.78
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.1)]) 1.96
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) 1.32
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.5)]) 0.68
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) -0.3
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.9)]) 0.12
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.1)]) 2.08
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.30000000000000004)]) 1.38
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('g

OrderedDict([('alpha', 0.9), ('beta', 0.1), ('gamma', 0.1)]) 0.64
OrderedDict([('alpha', 0.9), ('beta', 0.1), ('gamma', 0.30000000000000004)]) 0.02
OrderedDict([('alpha', 0.9), ('beta', 0.1), ('gamma', 0.5)]) 0.66
OrderedDict([('alpha', 0.9), ('beta', 0.1), ('gamma', 0.7000000000000001)]) 0.04
OrderedDict([('alpha', 0.9), ('beta', 0.1), ('gamma', 0.9)]) 0.68
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.1)]) 0.88
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) 0.3
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.5)]) -0.14
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) 0.42
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.9)]) 0.16
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.1)]) 1.44
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.30000000000000004)]) 0.78
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamm

In [16]:
sweep_size = 10
sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=sweep_size)
         * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=sweep_size)
         * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=sweep_size))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)

min = None
min_params = None
for result in results:
    value = obj_func(result)
    if min is None or value < min:
        min = value
        min_params = result.params
print(f'Minimum objective value is {min}.')

Minimum objective value is -1.7.
